# MovieLens рекомендации

In [0]:
import zipfile
from collections import defaultdict, Counter
import datetime

from scipy import linalg
import scipy.sparse as sps
import numpy as np
import matplotlib.pyplot as plt

## Готовим данные

Выборка представляет собой тройки $(u, i, r_{u,i})$, где $u$ -- пользователь, $i$ -- фильм, $r_{u,i}$ -- рейтинг. Нормализуем рейтинги на отрезок $[0, 1]$.

In [0]:
# download data
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2019-06-06 22:52:23--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  16.6MB/s    in 0.3s    

2019-06-06 22:52:24 (16.6 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [0]:
%%time
# read data
movies = {} # id
users = {} # id
ratings = defaultdict(list) # user-id

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    # archive overview
    print "files in archive"
    print z.namelist()
    #print "README"
    #print z.read("ml-1m/README")
    print "movies"
    with z.open("ml-1m/movies.dat") as m:
        print m.readline(),
    print "ratings"
    with z.open("ml-1m/ratings.dat") as m:
        print m.readline(),
    print "users"
    with z.open("ml-1m/users.dat") as m:
        print m.readline(),
    
    # parse movies
    with z.open("ml-1m/movies.dat") as m:
        for line in m:
            MovieID, Title, Genres = line.strip().split("::")
            MovieID = long(MovieID)
            Genres = Genres.split("|")
            movies[MovieID] = {"Title": Title, "Genres": Genres}
    
    # parse users
    with z.open("ml-1m/users.dat") as m:
        fields = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
        for line in m:
            row = zip(fields, line.strip().split("::"))
            data = dict(row[1:])
            data["Occupation"] = int(data["Occupation"])
            users[long(row[0][1])] = data
    
    # parse ratings
    with z.open("ml-1m/ratings.dat") as m:
        for line in m:
            UserID, MovieID, Rating, Timestamp = line.strip().split("::")
            UserID = long(UserID)
            MovieID = long(MovieID)
            Rating = int(Rating)
            Timestamp = long(Timestamp)
            ratings[UserID].append((MovieID, Rating, datetime.datetime.fromtimestamp(Timestamp)))

files in archive
['ml-1m/', 'ml-1m/movies.dat', 'ml-1m/ratings.dat', 'ml-1m/README', 'ml-1m/users.dat']
movies
1::Toy Story (1995)::Animation|Children's|Comedy
ratings
1::1193::5::978300760
users
1::F::1::10::48067
CPU times: user 5.28 s, sys: 3.18 s, total: 8.46 s
Wall time: 8.46 s


In [0]:
# train - test split
import math
train_frac = 0.8
train = []
test = []
for u, itemList in ratings.iteritems():
    # itemList = [(i, r, t), ...]
    all = sorted(itemList, key=lambda x: x[2])
    thr = int(math.floor(len(all) * train_frac))
    train.extend(map(lambda x: (u, x[0], x[1] / 5.0), all[:thr]))
    test.extend(map(lambda x: (u, x[0], x[1] / 5.0), all[thr:]))
print "ratings in train:", len(train)
print "ratings in test:", len(test)

ratings in train: 797758
ratings in test: 202451


## Item-based CF

Имея матрицу user-item из оценок пользователей можно определить меру adjusted cosine similarity похожести товаров $i$ и $j$ как векторов в пространстве пользователей: 

$sim(i, j) = \frac{\sum_{u \in U} (r_{u,i} - \overline{r_u}) (r_{u,j} - \overline{r_u}) }{\sqrt{\sum_{u \in U} (r_{u,i} - \overline{r_u})^2} \sqrt{\sum_{u \in U} (r_{u,j} - \overline{r_u})^2}}$,

где $U$ -- множество всех пользователей, $\overline{r_u}$ -- средний рейтинг пользователя $u$.
    
Рейтинги для неизвестных фильмов считаются по следующей формуле:
 
$\hat{r}_{u,i} = \sum_{j: \  r_{u,j} \ne 0} sim(i, j) r_{u,j} \bigg/ \sum_{j: \  r_{u,j} \ne 0} sim(i, j)$
    
Такой подход называется item-based. Обратим внимание на то, что $sim(i, j) \in [-1, 1]$. Это может привести к делению на ноль или значениям $\hat{r}_{u,i}$ вне диапазона $[0, 1]$. Избавиться от этой проблемы можно, например, положив равными нулю отрицательные значения $sim(i, j)$.

In [0]:
%%time
# CF step 1
# user avg
userAverage = defaultdict(list)
for u, i, r in train:
    userAverage[u].append(r)
for u in userAverage.iterkeys():
    userAverage[u] = min(np.mean(userAverage[u]), 0.99)
userAverage = dict(userAverage)

# norms
itemNorms = defaultdict(float)
for u, i, r in train:
    itemNorms[i] += (r - userAverage[u]) ** 2
for i in itemNorms.iterkeys():
    itemNorms[i] = np.sqrt(itemNorms[i])
itemNorms = dict(itemNorms)

# new ratings
itemUser = sps.csc_matrix(
    ([(r - userAverage[u]) / (itemNorms[i] + 1e-6) for u, i, r in train],
    ([e[1] for e in train], [e[0] for e in train]))
)
# item - item sim
itemItemSimilarity = itemUser.dot(itemUser.transpose())

CPU times: user 3.53 s, sys: 140 ms, total: 3.67 s
Wall time: 3.68 s


In [0]:
# test cos
print np.min(itemItemSimilarity.todense())
print np.max(itemItemSimilarity.todense())
print (itemUser[1, :] * itemUser[14, :].transpose())[0, 0]
print itemItemSimilarity[1, 14]

-0.9999922465208185
0.9999998462832396
0.001901025652624686
0.001901025652624686


In [0]:
# do not take top K for every item, so delete < 0
itemItemSimilarity[itemItemSimilarity < 0] = 0

In [0]:
%%time
# CF step 2
trainByUser = defaultdict(list)
testByUser = defaultdict(list)
for u, i, r in train:
    trainByUser[u].append((i, r))
for u, i, r in test:
    testByUser[u].append((i, r))
    
mse = 0
counter = 0 
NUM = 7000
COUNT = 0
for u, l in trainByUser.items()[0:NUM]:
    userRatedItems = [e[0] for e in l]
    userRatedRatings = sps.csr_matrix([[e[1]] for e in l])
    similarItems = itemItemSimilarity[:, userRatedItems]
    recoms = similarItems.dot(userRatedRatings) / (similarItems.sum(axis=1) + 1e-6)
    tbu = testByUser[u]
    testItems = [e[0] for e in tbu]
    testRatings = np.array([[e[1]] for e in tbu])
    errors = np.square(recoms[testItems, :] - testRatings)
    mse += sum(errors)
    COUNT += len(errors)
    counter += 1
    if counter % 500 == 0:
        print counter
print mse / COUNT

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
[[0.0342228]]
CPU times: user 1min 7s, sys: 945 ms, total: 1min 8s
Wall time: 1min 8s


## ALS факторизация

В этом подходе оценка $r_{ui}$ пользователя $u$, поставленная фильму $i$, ищется как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $\mathbb{R}^K$ латентных признаков:

$$
	\hat{r}_{ui} = p_u^T q_i 
$$

Иными словами, модель находит пространство признаков, в котором мы описываем и фильмы и пользователей и в котором рейтинг является мерой близости между фильмами и пользователями.
	
Для настройки модели будем минимизировать следующую ошибку:
	$$
	\sum_{(u, i, r_{ui})} (r_{ui} - p_u^T q_i)^2 + \lambda_{p} p_u^T p_u + \lambda_{q} q_i^T q_i,
	$$
	где суммирование ведется по всем тройкам $(u, i, r_{ui})$ выборки, слагаемые с $\lambda_{p}$ и $\lambda_{q}$ добавлены для регуляризации.
	
В методе  ALS проводятся $N$ итераций, в рамках каждой итерации сначала оптимизируется $p$ при фиксированном $q$, затем $q$ при фиксированном $p$.
	
Составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in \mathbb{R}^{n_u \times K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ -- количество оценок пользователя $u$.
	
Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке ridge-регрессии и выглядит так:
	$$
	A_u = Q[u]^T Q[u] \\
	d_u = Q[u]^T r_u \\
	p_u = (\lambda_p n_u I + A_u)^{-1} d_u
	$$
	
Формулы для перенастройки $q_i$ при фиксированной матрице $P$ выглядят аналогично.

In [0]:
%%time
LATENT_SIZE = 10
N_ITER = 20
lam_p = 0.2
lam_q = 0.001

n_users = max([e[0] for e in train]) + 1
n_items = max([e[1] for e in train]) + 1

p = 0.1 * np.random.random((n_users, LATENT_SIZE))
q = 0.1 * np.random.random((n_items, LATENT_SIZE))

ratedByUser = defaultdict(list)
for u, i, r in train:
    ratedByUser[u].append((i, r))
    
ratedByItem = defaultdict(list)
for u, i, r in train:
    ratedByItem[i].append((u, r))

def train_error(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in train])

def test_error(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in test])
    
for iter in range(N_ITER):
    for u, rated in ratedByUser.iteritems():
        ratedItems = [j for j, _ in rated]
        ratedScores = np.array([s for _, s in rated])
        Q = q[ratedItems, :]
        A = Q.transpose().dot(Q)
        d = Q.transpose().dot(ratedScores)
        p[u, :] = np.linalg.solve(lam_p * len(ratedItems) * np.eye(LATENT_SIZE) + A, d)
    for i, rated in ratedByItem.iteritems():
        ratedUsers = [j for j, _ in rated]
        ratedScores = np.array([s for _, s in rated])
        P = p[ratedUsers, :]
        A = P.transpose().dot(P)
        d = P.transpose().dot(ratedScores)
        q[i, :] = np.linalg.solve(lam_q * len(ratedUsers) * np.eye(LATENT_SIZE) + A, d)
    predictions = p.dot(q.transpose())
    print iter, train_error(predictions), test_error(predictions)

0 0.03378055930882804 0.035712124988088685
1 0.03042918950354409 0.03333912193357494
2 0.026814012227120657 0.03152228821975026
3 0.02549154978543663 0.031131746259699368
4 0.02498126976097821 0.030922431935166514
5 0.024728539708377795 0.03078170152152855
6 0.02458374288241625 0.030686782853145442
7 0.024490695149671508 0.030619868840279577
8 0.024425860568585225 0.030570491943186798
9 0.024378458622142885 0.03053248593758612
10 0.02434296175259042 0.030502173859928223
11 0.024316115044486237 0.03047732557980402
12 0.024295698713775554 0.030456651663776722
13 0.024280089342611638 0.030439404323065352
14 0.024268111313652883 0.030425034737680278
15 0.024258930294940494 0.030413062091810893
16 0.02425195605669319 0.03040305959089044
17 0.024246757999584503 0.03039464633023862
18 0.024242992293977424 0.030387502970100348
19 0.024240363609183992 0.030381391457814773
CPU times: user 1min 5s, sys: 30.7 s, total: 1min 36s
Wall time: 56.1 s


## Content-based

В таком подходе рекомендательная система пытается найти фильмы на основе: характеристик фильмов (например, жанр, режиссер, год выхода), профиля каждого пользователя в терминах характеристик фильмов, характеристик пользователей (например, пол, профессия).
    
Для каждой пары $u, i$ необходимо придумать признаки $f^n_{u, i}$, основанные на профиле пользователя, собранном на обучении, и характеристиках пользователей и фильмов, известных даже для новых пользователей и фильмов.
    
Следующий набор признаков можно использовать для рекомендательной системы:
* $f^1_{u, i}$ -- категориальный признак, возраст пользователя
* $f^2_{u, i}$ -- категориальный признак, профессия пользователя
* $f^3_{u, i}$ -- набор булевых признаков, по одному на каждый жанр, к которому отнесен фильм
* $f^4_{u, i}$ -- категориальный признак, пол пользователя
* $f^5_{u, i}$ -- $(u_g \, \cdot \, m_g) / n_g$, где $u_g$ -- вектор средних оценок пользователя в пространстве жанров, $m_g$ -- булевый вектор для фильма в пространстве жанров, $n_g$ -- количество жанров, указанных для фильма
* $f^6_{u, i}$ -- средний рейтинг пользователя
* $f^7_{u, i}$ -- средний рейтинг фильма
* $f^8_{u, i}$ -- константный признак
    
Категориальные признаки необходимо закодировать набором булевых векторов, по одному на каждое значение признака. Полученные признаки обозначим как $\{g^n_{u,i}\}_{n = 1 .. N}$.
	
Далее предлагается искать рейтинг как линейную комбинацию числовых признаков:
    $$
    \hat{r}_{u,i} = \sum_{n = 1}^{N} g^n_{u, i} \theta_n
    $$
    
Для настройки весов предлагается воспользоваться Ridge-регрессией. Для проверки реализации предлагается воспользоваться $\lambda = 0.2$.

In [0]:
print users[1]
print movies[1]

{'Gender': 'F', 'Age': '1', 'Zip-code': '48067', 'Occupation': 10}
{'Genres': ['Animation', "Children's", 'Comedy'], 'Title': 'Toy Story (1995)'}


In [0]:
%%time
# content-based step 1
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western"
]
genres = {k: v for v, k in enumerate(genres)}

sex = ["F", "M"]
sex = {k: v for v, k in enumerate(sex)}

age = ["1", "18", "25", "35", "45", "50", "56"]
age = {k: v for v, k in enumerate(age)}

profileByUser = {}
for u, il in ratedByUser.iteritems():
    temp = [0] * len(genres)
    for g in range(len(genres)):
        ll = []
        for i, r in il:
            for mg in movies[i]["Genres"]:
                if genres[mg] == g:
                    ll.append(r)
        if ll != []:
            temp[g] = np.mean(ll)
        else:
            temp[g] = 0
    profileByUser[u] = temp
meanByUser = {}
for u, il in ratedByUser.iteritems():
    meanByUser[u] = np.mean([v for _, v in il])
meanByItem = {}
for i, ul in ratedByItem.iteritems():
    meanByItem[i] = np.mean([v for _, v in ul])

fs = []
for u, i, r in train:
    f = []
    # age
    temp = [0] * len(age)
    temp[age[users[u]['Age']]] = 1
    f.extend(temp)
    # occ
    temp = [0] * 21
    temp[users[u]['Occupation']] = 1
    f.extend(temp)
    # movie genres
    temp = [0] * len(genres)
    for g in movies[i]["Genres"]:
        temp[genres[g]] = 1
    profileByItem = [t / len(movies[i]["Genres"]) for t in temp]
    f.extend(temp)
    # sex?
    temp = [0] * 2
    temp[sex[users[u]['Gender']]] = 1
    f.extend(temp)
    # fucking dot
    t = np.array(profileByUser[u]).dot(np.array(profileByItem))
    f.extend([t])
    # avg rate u
    f.extend([meanByUser[u]])
    # avg rate i
    f.extend([meanByItem[i]])
    # 1
    f.extend([1])
    fs.append(f)
fs = np.array(fs)

CPU times: user 25.5 s, sys: 1.63 s, total: 27.2 s
Wall time: 27.2 s


In [0]:
# test
fs[0, :]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.88571429, 0.83809524,
       0.69436202, 1.        ])

In [0]:
%%time
from sklearn.linear_model import Ridge
clf = Ridge(alpha=0.2)
y = np.array([[r] for _, _, r in train])
clf.fit(fs, y)
yy = clf.predict(fs)
print "train", np.mean(np.square(yy - y))

ffs = []
# test
for u, i, r in test:
    f = []
    # age
    temp = [0] * len(age)
    temp[age[users[u]['Age']]] = 1
    f.extend(temp)
    # occ
    temp = [0] * 21
    temp[users[u]['Occupation']] = 1
    f.extend(temp)
    # movie genres
    temp = [0] * len(genres)
    for g in movies[i]["Genres"]:
        temp[genres[g]] = 1
    profileByItem = [t / len(movies[i]["Genres"]) for t in temp]
    f.extend(temp)
    # sex?
    temp = [0] * 2
    temp[sex[users[u]['Gender']]] = 1
    f.extend(temp)
    # fucking dot
    t = np.array(profileByUser[u]).dot(np.array(profileByItem))
    f.extend([t])
    # avg rate u
    f.extend([meanByUser[u]])
    # avg rate i
    movr = 0
    if i in meanByItem:
        movr = meanByItem[i]
    f.extend([movr])
    # 1
    f.extend([1])
    ffs.append(f)
ffs = np.array(ffs)
y = np.array([[r] for _, _, r in test])
yyy = clf.predict(ffs)
print "test", np.mean(np.square(yyy - y))

train 0.03328043707497794
test 0.03497850859895244
CPU times: user 7.68 s, sys: 702 ms, total: 8.39 s
Wall time: 8.39 s


## ALS in Spark

In [0]:
# install spark (https://towardsdatascience.com/pyspark-in-google-colab-6821c2faf41c)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init('/content/spark-2.4.1-bin-hadoop2.7')
from pyspark import SparkContext
sc = SparkContext("local", "my app")

In [0]:
%%time
# ALS in spark mllib
from pyspark.mllib.recommendation import ALS, Rating

train = sc.parallelize(train)
test = sc.parallelize(test)
testPairs = test.map(lambda x: (x[0], x[1]))
trainPairs = train.map(lambda x: (x[0], x[1]))

rank = 10
numIterations = 20
model = ALS.train(train, rank, numIterations)

# performance on train set
predictions = model.predictAll(trainPairs).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = train.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).reduce(lambda x, y: x + y) / ratesAndPreds.count()
print("Mean Squared Error on train = " + str(MSE))

# performance on test set
predictions = model.predictAll(testPairs).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = test.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).reduce(lambda x, y: x + y) / ratesAndPreds.count()
print("Mean Squared Error on test = " + str(MSE))

Mean Squared Error on train = 0.0234785805574
Mean Squared Error on test = 0.0308082765621
CPU times: user 839 ms, sys: 92.5 ms, total: 932 ms
Wall time: 1min 28s
